In [2]:
import pandas as pd

In [5]:
df=pd.read_csv('../클러스터링고객데이터.csv')

In [5]:
df['이메일 (Email)'].unique().shape

(292,)

In [6]:
#연번 컬럼 삭제
df.drop('연번',axis=1,inplace=True)
df

,이름 (Name),생년월일 (Date of Birth),성별 (Gender),휴대폰번호 (Phone Number),이메일 (Email),주소 (Address),아이디 (User ID),가입일 (Registration Date),차량구분(vehicle types),구매한 제품 (Purchased Product),제품 구매 날짜 (Purchase Date),거래 금액 (Transaction Amount),거래 방식 (Transaction Method),제품 구매 빈도 (Purchase Frequency),제품 구매 경로 (Purchase Path),연령대,제품 출시년월 (Launch Date),Cluster
0,강주온,2002-03-04,남,010-6502-8753,janisi@himedia.co.kr,인천광역시 남동구 구월동 88-10,14798,2023-04-01,준중형 세단,Avante (CN7 N),2024-04-04,35170000,카드,1,오프라인,23,2020-05,8
1,김경수,1983-01-02,남,010-1111-2222,kimkyoungsoo31@himedia.co.kr,광주광역시 서구 치평동 14-7,kyoungsoo031,2025-02-09,준대형 세단,G80 (RG3),2025-02-10,85790000,카드,5,오프라인,42,2020-03,3
2,김다은,1999-05-02,여,010-6789-0123,kimdaeun26@himedia.co.kr,부산광역시 해운대구 우동 101-12,daeun026,2025-02-28,중형 suv,NEXO (FE),2025-03-10,69500000,현금,1,오프라인,26,2018-01,7
3,김민수,1960-07-01,남,010-1234-5678,kimminsu1@himedia.co.kr,전라남도 목포시 부주동 77-8,minsu001,2023-04-28,중형 suv,Santa-Fe ™,2023-05-01,34920000,현금,1,오프라인,65,2018-01,6
4,김승현,1970-03-02,남,010-0123-4567,kimseunghyun20@himedia.co.kr,경기도 수원시 영통구 원천동 12-34,seunghyun020,2023-09-18,중형 세단,G70 (IK),2023-10-01,50230000,카드,1,오프라인,55,2017-09,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,신시우,1958-07-11,여,042-207-6955,gyeonghyi78@gmail.com,전라북도 전주시 완산구 덕진동 33-7,jihungim,2023-03-06,준중형 세단,G80 (RG3),2024-12-20,86770000,카드,2,온라인,67,2020-03,4
288,최춘자,1976-06-27,여,031-170-7739,hancunja@yu.kr,충청남도 천안시 동남구 성성동 22-3,jinho50,2024-08-26,준대형 세단,Avante (CN7 N),2023-08-24,69510000,현금,3,오프라인,49,2020-05,0
289,차서준,1968-01-22,여,064-741-3740,qgim@simryujo.com,경상남도 창원시 의창구 봉곡동 48-6,jihun41,2023-01-02,준대형 세단,NEXO (FE),2024-01-12,39890000,현금,2,온라인,57,2018-01,6
290,안정희,1984-01-10,남,051-406-1803,donghyeonbae@joyunson.com,대구광역시 수성구 범어동 55-3,yeongil47,2023-04-08,소형 해치백,G80 (RG3),2024-01-30,44530000,현금,2,온라인,41,2020-03,6


In [7]:
df.to_csv('../클러스터링고객데이터.csv',index=False)

In [18]:
#모델 불러와서 예측해보기. 클러스터링과 클래시파이어 파이프라인 모델임
import joblib
model=joblib.load('../model.pkl')

In [31]:
new_data= [24,250000,2,'여','준중형 세단','카드','2016-01','2024-04-04']

In [33]:
new_data = np.array(new_data).reshape(1,8)

In [34]:
new_data

array([['24', '250000', '2', '여', '준중형 세단', '카드', '2016-01',
        '2024-04-04']], dtype='<U21')

In [ ]:
new_data = pd.DataFrame(new_data, columns=feature_columns)

NameError: name 'new_date' is not defined

In [30]:
import numpy as np

# new_data 값
new_data = [24, 250000, 2, '여', '준중형 세단', '온라인', '2016-01', '2024-04-04']

# 적절한 데이터 타입으로 변환
new_data_corrected = [
    int(new_data[0]),          # age
    int(new_data[1]),          # income
    int(new_data[2]),          # num_of_dependents
    new_data[3],               # gender
    new_data[4],               # car_type
    new_data[5],               # purchase_method
    pd.to_datetime(new_data[6]), # start_date
    pd.to_datetime(new_data[7])  # end_date
]

# numpy 배열로 변환
new_data_array = np.array(new_data_corrected).reshape(1, 8)

print(new_data_array)


[[24 250000 2 '여' '준중형 세단' '온라인' Timestamp('2016-01-01 00:00:00')
  Timestamp('2024-04-04 00:00:00')]]


In [22]:
import pandas as pd

# 컬럼 이름 지정
columns = ['age', 'income', 'num_of_dependents', 'gender', 'car_type', 'purchase_method', 'start_date', 'end_date']

# new_data를 데이터프레임으로 변환
df = pd.DataFrame([new_data], columns=columns)

# 모델 예측
model.predict(df)



ValueError: columns are missing: {'제품 출시년월 (Launch Date)', '거래 금액 (Transaction Amount)', '연령대', '제품 구매 빈도 (Purchase Frequency)', '차량구분(vehicle types)', '성별 (Gender)', '거래 방식 (Transaction Method)', '제품 구매 날짜 (Purchase Date)'}

In [23]:
import numpy as np

# new_data를 2차원 배열로 변환
new_data_2d = np.array(new_data).reshape(1, -1)

# 모델 예측
model.predict(new_data_2d)


ValueError: Specifying the columns using strings is only supported for dataframes.

In [24]:
new_data_2d

array([['24', '250000', '2', '여', '준중형 세단', '온라인', '2016-01',
        '2024-04-04']], dtype='<U21')